# Web Scraping Assignment-3**********

In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
import re 
import time
import pandas as pd
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import NoSuchElementException
import warnings
warnings.filterwarnings('ignore')

# 1. Write a python program which searches all the product under a particular product from www.amazon.in. The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [32]:
driver=webdriver.Chrome('chromedriver.exe')

In [33]:
driver.get("https://www.amazon.in/")

In [34]:
driver.maximize_window()
time.sleep(2)

In [35]:
product=input("Enter the Product Name: ")

Enter the Product Name: laptop


In [36]:
print("Product is: ",product)

Product is:  laptop


In [37]:
Input= driver.find_element(By.ID,'twotabsearchtextbox')
Input.send_keys(product)

In [38]:
Search = driver.find_element(By.ID,'nav-search-submit-button').click()

# 2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product name. Details to be scraped are: "Brand Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [45]:
#fetching URL to collect all the Product URLS 
product_url=[]
start=0
end=2
for page in range(start,end):
    url=driver.find_elements(By.XPATH,"//a[@class='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal']")
    for i in url:
        product_url.append(i.get_attribute('href'))
    #next_button=driver.find_elements("xpath",'//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]').click()
    next_4=driver.find_element("xpath",'/html/body/div[1]/div[2]/div[1]/div[1]/div/span[1]/div[1]/div[3]/div/div/div/div/div/div/div/div[2]/div/div/div[1]/h2/a').click()

In [46]:
len(product_url)

62

In [47]:
#Creating empty list 
Brand_name= []
Product_name = []
price=[]
Return_exc = []
Expected_Delivery = []
Availability = []

for i in product_url:
    driver.get(i)
    time.sleep(2)
    
    #Fetching data for Brand name    
    try:
        Brand=driver.find_element(By.XPATH,"//span[@class='a-size-large product-title-word-break']")
        Brand_name.append(Brand.text.split()[0]+" "+Brand.text.split()[1])
    except NoSuchElementException as e:
        Brand_name.append('-')
        
    #Fetching data for Product name        
    try:
        Product=driver.find_element(By.XPATH,"//span[@class='a-size-large product-title-word-break']")
        Product_name.append(Product.text)
    except NoSuchElementException as e:
        Product_name.append('-')
        
    #Fetching data for Product's Price
    try:
        price_tag=driver.find_element(By.XPATH,"//span[@class='a-price aok-align-center reinventPricePriceToPayMargin priceToPay']")
        price.append(price_tag.text.split("\n")[0])
    except NoSuchElementException as e:
        price.append('-')

    #Fetching data for Return/Exchange 
    try:
        Return_tag=driver.find_element(By.XPATH,"//div[@id='RETURNS_POLICY']/span/div[2]/a")
        Return_exc.append(Return_tag.text)
    except NoSuchElementException as e:
        Return_exc.append('-')

    #Fetching data for Expected Delivery 
    try:
        Delivery=driver.find_element(By.XPATH,"//div[@id='mir-layout-DELIVERY_BLOCK-slot-PRIMARY_DELIVERY_MESSAGE_LARGE']")
        Expected_Delivery.append(Delivery.text.split('delivery')[1].replace("Details"," "))
    except NoSuchElementException as e:
        Expected_Delivery.append('-')

    #Fetching data for Expected Delivery 
    try:
        Available=driver.find_element(By.XPATH,"//div[@id='availability']")
        Availability.append(Available.text)
    except NoSuchElementException as e:
        Availability.append('-')

In [48]:
print(len(Brand_name),len(Product_name),len(price),len(Return_exc),len(Expected_Delivery),len(Availability),len(product_url))

62 62 62 62 62 62 62


In [51]:
Output=pd.DataFrame({"Brand_name":Brand_name,"Product Name":Product_name,"Product Price":price,"Return/Exchange":Return_exc,
                       "Expected Delivery":Expected_Delivery,"Availability":Availability, "Product URL":product_url})
Output



,Brand_name,Product Name,Product Price,Return/Exchange,Expected Delivery,Availability,Product URL
0,Lenovo IdeaPad,Lenovo IdeaPad Slim 3 Intel Core i5 11th Gen 1...,"₹50,777",7 days Replacement,"Monday, 30 January. Order within 9 hrs 44 min...",In stock.,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
1,Lenovo Yoga,Lenovo Yoga Slim 7 Pro Intel Evo Core i5-11320...,"₹69,417",7 days Replacement,"Monday, 30 January. Order within 9 hrs 43 min...",In stock.,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
2,HP 15s-Ryzen,HP 15s-Ryzen 3 3250U 8GB SDRAM/256GB SSD 15.6i...,"₹28,106",7 days Replacement,"Wednesday, 1 February. Order within 23 hrs 43...",In stock.,https://www.amazon.in/HP-15s-Ryzen-15-6inch-Mi...
3,(Renewed) Dell,(Renewed) Dell Latitude E5470 Intel Core i5 6t...,-,7 days Replacement,"Tuesday, 31 January. Order within 15 hrs 28 m...",In stock.,https://www.amazon.in/Renewed-Dell-Latitude-La...
4,ASUS VivoBook,"ASUS VivoBook 15 (2021), 15.6-inch (39.62 cm) ...","₹24,827",7 days Replacement,"Wednesday, 1 February.",In stock.,https://www.amazon.in/ASUS-15-6-inch-Integrate...
...,...,...,...,...,...,...,...
57,HP ProBook,"HP ProBook 440 G8 Notebook PC, 11th Gen Intel ...","₹52,990",7 days Replacement,"Tuesday, 31 January.",In stock.,https://www.amazon.in/HP-ProBook-440-G8-i5-113...
58,"HP 14s,","HP 14s, 11th Gen Intel Core i3-1115G4, 8GB RAM...","₹35,990",7 days Replacement,"Tuesday, 31 January.",In stock.,https://www.amazon.in/HP-i3-1115G4-Micro-Edge-...
59,AVITA SATUS,AVITA SATUS S111 NU14A1INC43PN-SG 14.1 FHD (35...,"₹16,990",7 days Replacement,"Wednesday, 1 February. Order within 10 hrs 50...",In stock.,https://www.amazon.in/AVITA-SATUS-S111-NU14A1I...
60,Apple 2021,"Apple 2021 MacBook Pro (16-inch/41.05 cm, M1 M...","₹3,06,990",-,"Wednesday, 1 February. Order within 23 hrs 34...",In stock.,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...


In [52]:
#saving to csv file
Output_CSV=Output.to_csv('laptop.csv')
Output_CSV

# 3. A python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [60]:
# Opening chrome browser
webdriver.Chrome("chromedriver.exe")

# maximizing the browser size
driver.maximize_window()

# getting website "images.google.com"
url = "https://images.google.com/"
time.sleep(2)
    
#Creating empty list and giving search items as list and creating loop
urls = []    
data = []
search_item = ["fruits", "cars", "Machine Learning","guitar","cakes"]
for item in search_item:
    driver3.get(url)  
    time.sleep(5)
    search_img=driver3.find_element(By.XPATH,'//input[@class="gLFyf"]').send_keys(item)  #sending key word for search item
    
    search_btn = driver3.find_element(By.XPATH,'//button[@class="Tg7LZd"]').submit() #Clicking on search button
    
    # scrolling the web page to get more images
    for _ in range(20):
        driver3.execute_script("window.scrollBy(0,1000)")
        
        imgs = driver.find_elements(By.XPATH,"//img[@class='rg_i Q4LuWd']")
    img_url = []
    for image in imgs:
        source = image.get_attribute('src')
        if source is not None:
                if(source[0:4] == 'http'):
                    img_url.append(source)
    for i in img_url[:10]:
        urls.append(i)
                    
for i in range(len(urls)):
    if i > 10:
        break
    print("Downloading {0} of {1} images" .format(i, 10))
    response = requests.get(urls[i])
    

# 4. A python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [62]:
driver4=webdriver.Chrome('chromedriver.exe')
driver4.get("https://www.flipkart.com/")
time.sleep(2)
driver4.maximize_window()
pop_close = driver4.find_element("xpath","/html/body/div[2]/div/div/button").click()
product = input(" Enter the name of Smartphone: ")
search_product4=driver4.find_element(By.CLASS_NAME,'_3704LK').send_keys(product)

 Enter the name of Smartphone: Oneplus Nord


In [63]:
search_= driver4.find_element(By.CLASS_NAME,'L0Z3Pu').click()
  

In [64]:
page1_urls = []
urls = driver4.find_elements(By.XPATH,'//a[@class="_1fQZEK"]')
for url in urls:
    page1_urls.append(url.get_attribute("href"))

In [65]:
#  creating  empty  list
Brand = []
Phone_name = []
Colour = []
RAM= []
ROM = []
Primary_Camera= []
Secondary_Camera = []
Display_Size= []
Battery_Capacity = []
Price = []
URL = []

In [66]:
# Scraping data from each url of page 1
for url in page1_urls:
    driver4.get(url)              
    URL.append(url)                                                          
    time.sleep(2)
    
    
    #Clicking on read more button
    try:
        read_more = driver4.find_element(By.XPATH,'//button[@class="_2KpZ6l _1FH0tX"]')     
        read_more.click()
    except NoSuchElementException:
        print("Exception occured while moving to next page")
        
    
    #Scraping brand name of phone data
    try:
        brand_tags = driver4.find_element(By.XPATH,'//span[@class="B_NuCI"]')      
        Brand.append(brand_tags.text.split()[0])
    except NoSuchElementException:
        Brand.append('-')
    
    
    #Scraping phone name data
    try:
        name_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][1]/table/tbody/tr[3]/td[2]/ul/li')     
        Phone_name.append(name_tags.text)
    except NoSuchElementException:
        Phone_name.append('-')
    
    
    #Scraping phone color data
    try:
        color_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][1]/table/tbody/tr[4]/td[2]/ul/li')      
        Colour.append(color_tags.text)
    except NoSuchElementException:
        Colour.append('-')
     
    
    #Scraping RAM data
    try:
        ram_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[2]/td[2]/ul/li')                
        RAM.append(ram_tags.text)
    except NoSuchElementException:
        RAM.append('-')
    
    
    #Scraping ROM data
    try:
        rom_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[1]/td[2]/ul/li')        
        ROM.append(rom_tags.text)
    except NoSuchElementException:
        ROM.append('-')
        
        
    #Scraping Primary camera data
    try:                                                                                    
        pri_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[2]/td[2]/ul/li')
        Primary_Camera.append(pri_tags.text)
    except NoSuchElementException:
        Primary_Camera.append('-')
        
        
    #Scraping secondary camera data
    try:                                                                                    
        sec_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[1]')
        if sec_tags != "Secondary Camera" : 
            if driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[1]').text == "Secondary Camera":
                sec_cam = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[2]/ul/li')
            else :
                raise NoSuchElementException
        else :
            sec_cam = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[2]/ul/li')
        Secondary_Camera.append(sec_cam.text)
    except NoSuchElementException:
        Secondary_Camera.append('-')
        
     #Scraping Display size data 
    try:
        disp_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][2]/div')
        if disp_tags.text != "Displa Size" : raise NoSuchElementException
        disp_size = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][2]/table[1]/tbody/tr[1]/td[2]/ul/li')  
        Display_Size.append(disp_size.text)
    except NoSuchElementException:
        Display_Size.append('-')
        
        #Scraping battery capacity data
    try:
        if driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][10]/div').text != "Battery & Power Features" :
            if driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][9]/div').text == "Battery & Power Features" :
                bat_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[1]')
                if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
                bat_capa = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[2]/ul/li')                
            elif driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][8]/div').text == "Battery & Power Features" :
                bat_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[1]')
                if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
                bat_capa = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[2]/ul/li')
            else:
                raise NoSuchElementException
        else :
            bat_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[1]')
            if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
            bat_capa = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[2]/ul/li')              
        Battery_Capacity.append(bat_capa.text)
    except NoSuchElementException:
        Battery_Capacity.append('-')        
        
    #Scraping Price data
    try:
        price_tags = driver4.find_element(By.XPATH,'//div[@class="_30jeq3 _16Jk6d"]')      
        Price.append(price_tags.text)
    except NoSuchElementException:
        Price.append('-')

In [67]:
output=pd.DataFrame({"brand_name":Brand[:24],"smartphone_name":Phone_name[:24],"product_colour":Colour[:24],"product_RAM":RAM[:24],
                         "product_ROM":ROM[:24],"Primary_Camera":Primary_Camera[:24],"Secondary_Camera":Secondary_Camera[:24],
                         "Display_size":Display_Size[:24],"Battery_Capacity":Battery_Capacity[:24],"Phone_Price":Price[:24],"Product_URL":URL[:24]})

output

,brand_name,smartphone_name,product_colour,product_RAM,product_ROM,Primary_Camera,Secondary_Camera,Display_size,Battery_Capacity,Phone_Price,Product_URL
0,OnePlus,Nord CE 2 Lite 5G,Black Dusk,6 GB,128 GB,-,-,-,-,"₹19,930",https://www.flipkart.com/oneplus-nord-ce-2-lit...
1,OnePlus,Nord CE 2 Lite 5G,Blue Tide,6 GB,128 GB,Primary & Secondary Camera,-,-,-,"₹19,996",https://www.flipkart.com/oneplus-nord-ce-2-lit...
2,OnePlus,Nord CE 2 Lite 5G,Blue Tide,6 GB,128 GB,-,-,-,-,"₹19,990",https://www.flipkart.com/oneplus-nord-ce-2-lit...
3,OnePlus,Nord CE 2 Lite 5G,Blue Tide,6 GB,128 GB,Primary Camera,-,-,-,"₹19,900",https://www.flipkart.com/oneplus-nord-ce-2-lit...
4,OnePlus,Nord CE 2 5G,Bahama Blue,8 GB,128 GB,Primary Camera,-,-,-,"₹24,998",https://www.flipkart.com/oneplus-nord-ce-2-5g-...
5,OnePlus,Nord CE 2 5G,Bahama Blue,8 GB,128 GB,Primary Camera,-,-,-,"₹24,499",https://www.flipkart.com/oneplus-nord-ce-2-5g-...
6,OnePlus,Nord 2 5G,Gray Sierra,8 GB,128 GB,Primary Camera,-,-,-,"₹29,999",https://www.flipkart.com/oneplus-nord-2-5g-gra...
7,OnePlus,Nord CE 2 Lite 5G,Black Dusk,6 GB,128 GB,Primary & Secondary Camera,-,-,-,"₹19,900",https://www.flipkart.com/oneplus-nord-ce-2-lit...
8,OnePlus,Nord 2 5G,Blue Haze,12 GB,256 GB,Primary Camera,-,-,-,"₹33,998",https://www.flipkart.com/oneplus-nord-2-5g-blu...
9,OnePlus,Nord 2 5G,Blue Haze,8 GB,128 GB,Primary Camera,-,-,-,"₹29,799",https://www.flipkart.com/oneplus-nord-2-5g-blu...


In [68]:
output_csv=output.to_csv('smart_phone.csv')
output_csv

# 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [71]:
driver_=webdriver.Chrome("chromedriver.exe") 
time.sleep(2)

# opening google maps web page
driver_.get("https://www.google.co.in/maps")
driver_.maximize_window()
time.sleep(2)

#Sending keyword for seach box and search button
city = input('Enter City name: ')
search_city = driver_.find_element(By.ID,"searchboxinput")                       
search_city.clear()                                                             
time.sleep(2)

search_city.send_keys(city)                                                     
search_button = driver_.find_element(By.ID,"searchbox-searchbutton").click()                                                             
time.sleep(3)
Location=driver_.current_url

try:
    coordinates=re.findall(r'@(.*)data',Location)
    lat_lon=coordinates[0].split(",")
    lat=lat_lon[0]
    lon=lat_lon[1]
except IndexError as e:
    coordinates=re.findall(r'@(.*)z',Location)
    lat_lon=coordinates[0].split(",")
    lat=lat_lon[0]
    lon=lat_lon[1]
print("\nlatitude={}".format(lat))
print("\nlongitude={}".format(lon))

Enter City name: Bangalore

latitude=12.9542946

longitude=77.4908525


# Question No. 6 is skipped for now as informed.

# 7.Write a program to scrap all the available details of best gaming laptops from digit.in.

In [113]:
driver=webdriver.Chrome("chromedriver.exe") 
time.sleep(2)

# opening Digit.in web page
driver.get("https://digit.in/")
driver.maximize_window()
time.sleep(2)

In [114]:
driver.find_element(By.XPATH,"/html/body/div[4]/div/div[2]/div[2]/div[4]/ul/li[9]/a").click()

In [115]:
product_name=[]
seller=[]
price=[]

product_tag=driver.find_elements(By.XPATH,"//*[@id='summtable']/tbody/tr/td[1]")
for i in product_tag:
    try:
        product_name.append(i.text)
    except NoSuchElementException:
        product_name.append("-")

seller_tag=driver.find_elements(By.CLASS_NAME,"smmerchant")
for i in seller_tag:
    try:
        seller.append(i.text)
    except NoSuchElementException:
        seller.append("-")

price_tag=driver.find_elements(By.CLASS_NAME,"smprice")
for i in price_tag:
    try:
        price.append(i.text)
    except NoSuchElementException:
        price.append("-")



In [117]:
Output=pd.DataFrame({"Product Name":product_name,"Seller":seller,"Price":price})
Output

,Product Name,Seller,Price
0,MSI Titan GT77-12UHS,N/A,N/A
1,Alienware x17 R2,Croma,"₹ 389,990"
2,Acer Predator Triton 500 SE PT516-52s,Amazon,"₹ 229,990"
3,Omen by HP (16-B1371TX),Amazon,"₹ 176,490"
4,Acer Predator Helios 300 AN515-45,N/A,"₹ 172,999"
5,MSI Delta 15 (A5EFK-083IN),N/A,"₹ 188,990"
6,Omen by HP (16-c0141AX),Croma,"₹ 124,990"
7,Lenovo Legion 5i Pro (82RF00MGIN),Amazon,"₹ 167,990"
8,Alienware m15 R5 Ryzen Edition icc-c780001win,Amazon,"₹ 174,000"
9,Lenovo Slim 7 Gen 6 (82K8002JIN),Croma,"₹ 131,990"


# 8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”

In [118]:
driver=webdriver.Chrome("chromedriver.exe") 
time.sleep(2)

# opening web page
driver.get("https://www.forbes.com/billionaires/")
driver.maximize_window()
time.sleep(2)

In [120]:
# Creating Empty List:
Rank = []
Name = []
Net_worth = []
Age = []
Citizen = []
Source = []
Industry = []


rank_ = driver.find_elements(By.XPATH,"//div[@class='rank']")
for r in rank_:
    try:
        Rank.append(r.text)
    except NoSuchElementException:
        Rank.append("-")
    

name_ = driver.find_elements(By.XPATH,"//div[@class='personName']")
for n in name_:
    try:
        Name.append(n.text)
    except NoSuchElementException:
        Name.append("-")
        

networth_ = driver.find_elements(By.XPATH,"//div[@class='netWorth']")
for nt in networth_:
    try:
        Net_worth.append(nt.text)
    except NoSuchElementException:
        Net_worth.append("-")
        

age_ = driver.find_elements(By.XPATH,"//div[@class='age']")
for a in age_:
    try:
        Age.append(a.text)
    except NoSuchElementException:
        Age.append("-")
    

citizen_= driver.find_elements(By.XPATH,"//div[@class='countryOfCitizenship']")
for c in citizen_:
    try:
        Citizen.append(c.text)
    except NoSuchElementException:
        Citizen.append("-")
    

src_ = driver.find_elements(By.XPATH,"//span[@class='source-text']")
for s in src_:
    try:
        Source.append(s.text)
    except NoSuchElementException:
        Source.append("-")
    
#Fetching Industry
industry_ = driver.find_elements(By.XPATH,"//div[@class='category']")
for ind in industry_:
        try:
            Industry.append(ind.text)
        except NoSuchElementException:
            Industry.append("-")
        

In [121]:
Output =pd.DataFrame({'Rank':Rank,'Name':Name,'Net Worth':Net_worth,'Age': Age,'Citizenship/Country':Citizen,'Source':Source,'Industry':Industry})
Output

,Rank,Name,Net Worth,Age,Citizenship/Country,Source,Industry
0,1.,Elon Musk,$219 B,50,United States,"Tesla, SpaceX",Automotive
1,2.,Jeff Bezos,$171 B,58,United States,Amazon,Technology
2,3.,Bernard Arnault & family,$158 B,73,France,LVMH,Fashion & Retail
3,4.,Bill Gates,$129 B,66,United States,Microsoft,Technology
4,5.,Warren Buffett,$118 B,91,United States,Berkshire Hathaway,Finance & Investments
...,...,...,...,...,...,...,...
195,192.,Marcel Herrmann Telles,$10.3 B,72,Brazil,beer,Food & Beverage
196,197.,Leon Black,$10 B,70,United States,private equity,Finance & Investments
197,197.,Joe Gebbia,$10 B,40,United States,Airbnb,Technology
198,197.,David Geffen,$10 B,79,United States,"movies, record labels",Media & Entertainment


# 9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [122]:
driver=webdriver.Chrome('chromedriver.exe')
driver.get("https://www.youtube.com/watch?v=USccSZnS8MQ")
driver.maximize_window()
time.sleep(2)

In [123]:
i=0
while(i<100):
    driver.execute_script("window.scrollBy(0,500)") # scroll down to get more comments
    i+=1
while(i<402):
    driver.execute_script("window.scrollBy(0,5000)") # scroll down to get more comments
    i+=1
    
Comment = []
Upvote = []
Comment_time = []

comment_=driver.find_elements(By.XPATH,'//yt-formatted-string[@id="content-text"]')
for i in comment_:             
    try:
        Comment.append(i.text)
    except NoSuchElementException:
        Comment.append("-")

upvote_=driver.find_elements(By.XPATH,"//*[@id='vote-count-middle']")
for i in upvote_:             
    try:
        Upvote.append(i.text)
    except NoSuchElementException:
        Upvote.append("-")

comment_time_=driver.find_elements(By.XPATH,"//*[@id='header-author']/yt-formatted-string/a")
for i in comment_time_:             
    try:
        Comment_time.append(i.text)
    except NoSuchElementException:
        Comment_time.append("-")

In [128]:
len(Comment) ,len(Upvote),len(Comment_time)

(240, 260, 260)

In [129]:
Output_=pd.DataFrame({"Comment":Comment[:240],"Upvote":Upvote[:240],"Comment_ Time":Comment_time[:240]})
Output_


,Comment,Upvote,Comment_ Time
0,It's 2023 still the magic of this song never b...,154,3 weeks ago
1,A message to the future generations..Don’t let...,432,4 months ago
2,This song never gets old. No matter how much I...,67,2 weeks ago
3,My ears : listening the music\n\nMy lips : sin...,224,1 month ago
4,"Hi. I'm an American singer, Jason Ray. The sto...",69K,1 year ago
...,...,...,...
235,Being single whole life and having one sided l...,1,3 months ago (edited)
236,Amazing song,1,1 month ago (edited)
237,495M views,,1 month ago
238,I love ️ this SONG ️,1,13 days ago


# 10.  Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [2]:
driver0=webdriver.Chrome('chromedriver.exe')
driver0.get("https://www.hostelworld.com/")
driver0.maximize_window()
time.sleep(2)


In [4]:
inp=driver0.find_element(By.XPATH,'//input[@class="search-input"]').send_keys('london')
time.sleep(2)



In [6]:
# Make empty lists
Hostel_Name = []
Distance = []
overall_review = []
total_reviews = []
facilities = []
price = []
Rating = []
property_description = []


In [7]:
while(True):
    
    # Hostel name
    names = driver0.find_elements(By.XPATH,'//h2[@class="title title-6"]')
    for name in names:
        Hostel_Name.append(name.text)
    time.sleep(2)
        
    # Distance from city
    distance = driver0.find_elements(By.XPATH,'//span[@class="description"]')
    for d in distance:
        Distance.append(d.text)
    time.sleep(2)
        
    #Review    
    review = driver0.find_elements(By.XPATH,'//div[@class="keyword"]//span')
    for r in review:
        overall_review.append(r.text)
    time.sleep(2)
    
    # Total number of reviews     
    t_review = driver0.find_elements(By.XPATH,'//div[@class="reviews"]')
    for t in t_review:
        total_reviews.append(t.text)
    time.sleep(2)
    
    # facilities
    service = driver0.find_elements(By.XPATH,'//div[@class="facilities-label facilities"]')
    for s in service:
        facilities.append(s.text)
    time.sleep(2)
    
    # Prices    
    prices = driver0.find_elements(By.XPATH,'//div[@class="price-col"]')
    for p in prices:
        price.append(p.text)
    time.sleep(2)    
    try:
        next_button = driver0.find_element(By.XPATH,'//div[@class="pagination-item pagination-next"]')
        next_button.click()
    except:
        break
        
time.sleep(2) 

# Separate  Privates_From price  and  Dorms_From price
private = []
for i in range(0,len(price),2):
    private.append(price[i])
time.sleep(2)

dorms = []
for i in range(1,len(price),2):
    dorms.append(price[i]) 

In [8]:
# scrape Hostels URL
hostel_url = []

while(True):
    urls = driver0.find_elements(By.XPATH,'//h2[@class="title title-6"]/a')
    for url in urls:
        hostel_url.append(url.get_attribute("href"))
    time.sleep(2)    
        
    try:
        next_button = driver0.find_element(By.XPATH,'//div[@class="pagination-item pagination-prev"]')
        next_button.click()
    except:
        break
        
Rate = []
for page in hostel_url:
    driver0.get(page)
    
    # Rating
    try:
        ratings = driver0.find_element(By.XPATH,'//*[@id="__layout"]/div/div[1]/section/div[6]/div/div[1]/div[1]/div[1]')
        Rate.append(ratings.text)
    except NoSuchElementException:
        Rate.append("No Rating")  
    time.sleep(2)
    
    # Property Description
    try:
        pd = driver0.find_element(By.XPATH,'//*[@id="__layout"]/div/div[1]/section/div[6]/div/div[2]/div[2]/div/div[2]')
        property_description.append(pd.text)
    except NoSuchElementException:
        property_description.append("No Description")  

    
time.sleep(2)        
# remove extra data from Rating     
all_text = []
for i in Rate:
    all_text.append(i.split())
time.sleep(2)

for i in all_text:
    Rating.append(i[0]) 

In [10]:
print(
    len(Hostel_Name),len(Distance),len(Rating),len(total_reviews),len(overall_review),
    len(private),len(dorms),len(facilities),len(property_description))

30 30 30 30 32 30 30 30 30


In [11]:
Output=pd.DataFrame({"Hostel Name":Hostel_Name,"Distance":Distance,"Rating":Rating,"Total Reviews":total_reviews,"Overall Review":overall_review[:30],
                     "Private Price":private,"Dorms Price":dorms,"Facilities":facilities,"Property Description":property_description})
Output

,Hostel Name,Distance,Rating,Total Reviews,Overall Review,Private Price,Dorms Price,Facilities,Property Description
0,NX London Hostel,Hostel - 6.1km from city centre,No,1550 Total Reviews,Fabulous,Privates From\nRs6334.96 Rs4434,Dorms From\nRs1809.81 Rs1448,Free WiFi\nFree Breakfast,No Description
1,Wombat's City Hostel London,Hostel - 3.6km from city centre,No,14521 Total Reviews,Fabulous,Privates From\nRs13056.79 Rs10445,Dorms From\nRs3330.49 Rs2664,Free WiFi\nFollows Covid-19 sanitation guidance,No Description
2,St Christopher's Village,Hostel - 1.8km from city centre,No,11869 Total Reviews,Very Good,Privates From\nRs6129.62 Rs5517,Dorms From\nRs1576.56 Rs1419,Free WiFi\nFollows Covid-19 sanitation guidance,No Description
3,Astor Hyde Park,Hostel - 4.3km from city centre,No,11870 Total Reviews,Fabulous,Privates From\nRs9630,Dorms From\nRs1751,Free WiFi\nFollows Covid-19 sanitation guidance,No Description
4,Hostel One Notting Hill,Hostel - 5.5km from city centre,No,1718 Total Reviews,Fabulous,Privates From\nRs7896,Dorms From\nRs2449,Free WiFi,No Description
5,Pickwick Hall,Hostel - 2.3km from city centre,No,2554 Total Reviews,Fabulous,Privates From\nRs6061,No Dorms Available,Free Breakfast,No Description
6,Generator London,Hostel - 3km from city centre,No,7334 Total Reviews,Superb,Privates From\nRs7319,Dorms From\nRs2030,Free WiFi\nFollows Covid-19 sanitation guidance,No Description
7,Safestay London Elephant & Castle,Hostel - 1.7km from city centre,No,4798 Total Reviews,Fabulous,Privates From\nRs5455,Dorms From\nRs1546,Free WiFi\nFollows Covid-19 sanitation guidance,No Description
8,Urbany Hostel London,Hostel - 5.4km from city centre,No,630 Total Reviews,Very Good,Privates From\nRs7593,Dorms From\nRs2144,Free WiFi\nFollows Covid-19 sanitation guidance,No Description
9,Smart Hyde Park Inn Hostel,Hostel - 5km from city centre,No,6497 Total Reviews,Very Good,No Privates Available,Dorms From\nRs2330.03 Rs2097,Free WiFi,No Description
